# Mapreader Workshop - 30th April - 1st May 2024

----

First check you have the correct version of MapReader. 
For the April 2024 workshop, this is v1.2.0

This can be downloaded from pypi using `pip install mapreader==1.2.0` or by checking out the repo at [this commit](https://github.com/Living-with-machines/MapReader/releases/tag/v1.2.0)

In [1]:
import mapreader
assert mapreader.__version__ == '1.2.0'

-------------

# Download

https://mapreader.readthedocs.io/en/latest/User-guide/Download.html

MapReader accepts different types of map images as input. 

We're focusing on georeferenced maps that are available as tile layers from libraries (such as the [NLS](https://maps.nls.uk/)), archives, or other services.

MapReader’s ``Download`` subpackage is used to download maps stored as tile layers on a tile server. It contains two classes for downloading maps:

1. ``SheetDownloader`` - This can be used to download map sheets and relies on information provided in a metadata json file.

2. ``Downloader`` - This is used to download maps using polygons and can be used even if you don’t have a metadata json file.

In this workshop, we will use the ``SheetDownloader`` along with metadata from the NLS stored in the ``persistent_data`` directory of the mapreader repository.

## Set up the ``SheetDownloader``

In [ ]:
from mapreader import SheetDownloader

To get your download URL, please set up a free account on [maptiler](https://cloud.maptiler.com/maps/) and go to https://cloud.maptiler.com/tiles/uk-osgb10k1888/ to find the tile layer for the 2nd edition OS 6inch maps of the UK.

The URL you will need is the XYZ url listed under the Raster tiles heading.

__**YOUR TURN**__: Set up your ``my_ts`` by passing the ``metadata_path`` and ``download_url`` arguments to the ``SheetDownloader`` .

In [ ]:
metadata_path = "../NLS_metadata/metadata_OS_Six_Inch_GB_WFS_light.json"
download_url = "" # get this from maptiler

In [ ]:
# my_ts = SheetDownloader()

## Select maps to download

Your ``SheetDownloader`` instance (``my_ts``) can be used to query and download map sheets using a number of methods:

1. Any which are within or intersect/overlap with a polygon.
2.  Any which contain a set of given coordinates.
3. Any which intersect with a line.
4. By WFS ID numbers.
5. By searching for a string within a metadata field.

These methods can be used to either directly download maps or to create a list of queries which can interacted with and downloaded subsequently.

We will be using the querying option.

### 1. Finding map sheets which overlap or intersect with a polygon.

The ``.query_map_sheets_by_polygon()`` and ``.download_map_sheets_by_polygon()`` methods can be used find and download map sheets which are within or intersect/overlap with a [shapely.Polygon](https://shapely.readthedocs.io/en/stable/reference/shapely.Polygon.html#shapely.Polygon).

These methods have two modes:

- "within" - This finds map sheets whose bounds are completely within the given polygon.
- "intersects" - This finds map sheets which intersect/overlap with the given polygon.

The ``mode`` can be selected by specifying ``mode="within"`` or ``mode="intersects"``.

#### Create a polygon

In [ ]:
from mapreader import create_polygon_from_latlons

__**YOUR TURN**__: Create a polygon using the following latlons: ``55.65, -3.1, 55.7, -3``

In [ ]:
# my_polygon = create_polygon_from_latlons()

#### Find maps

To find map sheets which fall **within** the bounds of this polygon:

In [ ]:
my_ts.query_map_sheets_by_polygon(my_polygon, mode="within", print=True)

Nothing was found - our polygon is too small for any maps to fall completely within it.

Instead, to find map sheets which **intersect** with this polygon:

In [ ]:
my_ts.query_map_sheets_by_polygon(my_polygon, mode="intersects", print=True)

In [ ]:
len(my_ts.found_queries)

To see what you've found, plot your query results on a map using the ``.plot_queries_on_map()`` method:

In [ ]:
my_ts.extract_wfs_id_nos()
my_ts.plot_queries_on_map(map_extent="uk", add_id=False)

### 2. Finding map sheets which contain a set of coordinates.

The ``.query_map_sheets_by_coordinates()`` and ``.download_map_sheets_by_coordinates()`` methods can be used find and download map sheets which contain a set of coordinates.

> **_NOTE:_** We use the ``append=True`` argument to ensure our new queries are appended to our existing list.

__**YOUR TURN**__: Find maps which intersect with the following coordinates: ``(-4.5, 55.4)``

In [ ]:
# coords = 

In [ ]:
my_ts.query_map_sheets_by_coordinates(coords, print=True, append=True)

In [ ]:
len(my_ts.found_queries)

We've used the ``append=True`` argument and so, if you plot your found queries, you will see a new map sheet has been added to your queries list.

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk", add_id=False)

### 3. Finding map sheets which intersect with a line.

The ``.query_map_sheets_by_line()`` and ``.download_map_sheets_by_line()`` methods can be used find and download map sheets which intersect with a [shapely.LineString](https://shapely.readthedocs.io/en/stable/reference/shapely.LineString.html#shapely.LineString).

#### Create a line

In [ ]:
from mapreader import create_line_from_latlons

__**YOUR TURN**__: Create a line using the following latlons: ``(56.5, -5), (57.0, -4.5)``

In [ ]:
# my_line = create_line_from_latlons()

#### Find maps

> **_NOTE:_** In the previous examples, we used the ``print=True`` argument to print our query results each time. We've now removed this so query results aren't being printed.

In [ ]:
my_ts.query_map_sheets_by_line(my_line, append=True)

In [ ]:
len(my_ts.found_queries)

Again, after plotting your queries on a map, you'll see some new map sheets have been added to your queries list.

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk", add_id=False)

### 4. Finding map sheets using their WFS ID numbers.

The ``.query_map_sheets_by_wfs_ids()`` and ``.download_map_sheets_by_wfs_ids()`` methods can be used find and download map sheets using their WFS ID numbers.

These are the unique IDs for the WFS.

#### One map at a time

In [ ]:
my_ts.query_map_sheets_by_wfs_ids(12, append=True)

#### Multiple maps at a time

__**YOUR TURN**__: Find maps with the following WFS ids: ``[30, 37, 38]``

In [ ]:
# id_list = 

In [ ]:
my_ts.query_map_sheets_by_wfs_ids(id_list, append=True)

In [ ]:
len(my_ts.found_queries)

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk", add_id=False)

### 5. Finding map sheets by searching for a string in their metadata.

The ``.query_map_sheets_by_string()`` and ``.download_map_sheets_by_string()`` methods can be used find and download map sheets by searching for a string in their metadata.

These methods use [regex string searching](https://docs.python.org/3/library/re.html) to find map sheets whose metadata contains a given string. 
Wildcards and regular expressions can therefore be used in the ``string`` argument.

In [ ]:
my_ts.query_map_sheets_by_string("Stirling", append=True)

In [ ]:
len(my_ts.found_queries)

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk", add_id=False)

The above query command will search for "Stirling" in **all** metadata fields.

If instead, you'd like to search a particular metadata field (e.g. "IMAGEURL"), you can specify the ``keys`` argument.

> _**NOTE**_: You will need to have an idea of the structure of your metadata in order to do this. Use ``my_ts.features[0]`` to view the metadata for the first map sheet in our metadata, if you would like to see how our metadata is structured.

__**YOUR TURN**__: Explore the NLS metadata we are using for this workshop. You can do this either using the command above or by opening the json file found in ``"../NLS_metadata/metadata_OS_Six_Inch_GB_WFS_light.json"``

In [ ]:
# my_ts.features[0]

For the maps we are using in this workshop, it is possible to use the NLS Maps online collection to identify metadata strings to search for, like the words in the title or the unique ids that are present in the image URL. 

For example, if you navigate in a new browser window to https://maps.nls.uk/view/75650661, you can see one of the six-inch sheets. In MapReader, this can be selected by searching for "75650661" (the number at the end of the URL) in the "IMAGEURL" field.

To find other maps, you can use the index of digitized one-inch maps provided by the NLS [here](https://maps.nls.uk/os/6inch-england-and-wales/).

__**YOUR TURN**__: Find https://maps.nls.uk/view/75650661 by searching for ``"75650661"`` in the ``"IMAGEURL"`` field of the metadata (remember this is a nested under the ``"properties"`` field).

In [ ]:
# search_string = ""
# keys = []

In [ ]:
my_ts.query_map_sheets_by_string(
    search_string, 
    keys, 
    append=True,
)

In [ ]:
len(my_ts.found_queries)

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk", add_id=False)

### Print found queries

You can print your queries list at any time using the ``.print_found_queries()`` method.

This means you can run multiple queries and check what you've found at the end.

In [ ]:
my_ts.print_found_queries()

## Download query results

### Quick download

The cells above demonstrate the different ways we can specify which maps we are interested in.

But, for the workshop we do not want to spend a long time waiting for the maps to download. Therefore we will reset the query list with a shorter list before downloading the maps.

In [ ]:
# This is a preselected list of map sheets from the coast near Glasgow in Scotland
nls_image_url_ids = [
    "75650661",
    "75650697",
    "75650823",
    "75650907",
    "75650892",
    "75650931",
]

In [ ]:
my_ts.found_queries = []

In [ ]:
for nls_image_url_id in nls_image_url_ids:
    my_ts.query_map_sheets_by_string(
        nls_image_url_id, keys=["properties", "IMAGEURL"], append=True
    )

In [ ]:
len(my_ts.found_queries)

In [ ]:
my_ts.plot_queries_on_map(map_extent="uk", add_id=False)

To download the image files of the maps from the NLS tile server, you need to first set a zoom level using the ``.get_grid_bb()`` method.

We are using zoom level 17 - which will give us high resolution images.

In [ ]:
my_ts.get_grid_bb(17)

Then, you can download your map sheets using ``.download_map_sheets_by_queries()``:

> _**NOTE**_: We have left ``path_save`` and ``metadata_fname`` as the default values, so maps will be saved in ``"./maps/"`` and their metadata will be saved as ``"./maps/metadata.csv"``.

In [ ]:
my_ts.download_map_sheets_by_queries()

-----

# Load

MapReader’s ``Load`` subpackage is used to load, visualize and patchify images (e.g. maps) saved locally.

In this workshop, we will load the images that we have just downloaded from the NLS tile server.
These are saved in ``"./maps/"``.

However, you could use any images that you have saved locally. 
See the [Input Guidance](https://mapreader.readthedocs.io/en/latest/Input-guidance.html) section of the MapReader documentation for more details about file formats and metadata requirements.

## Import the ``loader`` and create your ``my_files`` object.

In [ ]:
from mapreader import loader

__**YOUR TURN**__: Load your maps. They are saved in ``"./maps/"`` as ``"*.png"`` files.

In [ ]:
# my_files = loader()

### Add metadata

Add the ``metadata.csv`` file that was created when downloading your maps.

__**YOUR TURN**__: Add your metadata. It is saved in ``"./maps/metadata.csv"``

In [ ]:
# my_files.add_metadata()

In [ ]:
parent_df, patch_df = my_files.convert_images()
parent_df.head()

## Patchify

The ``.patchify_all()`` method is used to slice your map images into patches. 

The method used to patchify your maps is selected by specifying ``method="pixel"`` or ``method="meters"``. This determines whether your ``patch_size`` is interpreted with units of ``pixel`` or ``meters``. 

#### Patchify by pixel

> _**NOTE**_: We have used ``add_to_parents=False`` here so that these patches are not added to the ``my_files`` object. This is simply so that we don't have two sets of patches added after running both ``.patchify_all()`` commands.

In [ ]:
my_files.patchify_all(method="pixel", patch_size=500, add_to_parents=False)

#### Patchify by meters

You will only be able to use ``method="meters"`` if you have coordinates saved for each of your map images. 
These coordinates should correspond to the "bounding box" of your map image (minimum x, minimum y, maximum x and maximum y) and thereby associate the left, bottom, right and top edges of your map image to their geospatial locations.

These can be added by running either ``.add_metadata()`` (assuming your metadata contains these coordinates) or, if your images contain georefencing information, ``.add_geo_info()``.

__**YOUR TURN**__: Patchify your maps into 100 meter patches.

In [ ]:
# my_files.patchify_all()

You will see your patches are saved in separate directories, each indicating the patch size and method used.

### Visualize results

``MapReader`` also contains some useful functions for visualizing your patches.

For example, the ``.show_sample()`` method can be used to show a random sample of your patches:

In [ ]:
my_files.show_sample(num_samples=3, tree_level="patch", random_seed=1)

You may also want to see all the patches created from one of your parent images. 
This can be done using the ``.show_parent()`` method:

In [ ]:
my_files.show_parent("map_75650661.png")

The ``.calc_pixel_stats()`` method can be used to calculate statistics on the pixel intensities of each patch.

Patches with higher pixel intensities are closer to white and so are more likely to contain blank space whereas patches with lower pixel intensities are more likely to contain features. Knowing pixel intensities can be useful for sorting patches when annotating. 

In [ ]:
my_files.calc_pixel_stats()

By running the ``.convert_images()`` method, you will see that the means and standard deviations of pixel intensities of each patch have been added to your ``my_files`` object.

In [ ]:
parent_df, patch_df = my_files.convert_images()
patch_df.head()

To save these outputs, use the ``save=True`` argument.

> _**NOTE**_: By default, this will save your outputs as ``.csv`` files. If instead, you'd like to save as ``.xslx`` files, add ``save_format="excel"`` to your command.

In [ ]:
parent_df, patch_df = my_files.convert_images(save=True)

In [ ]:
parent_df, patch_df = my_files.convert_images(save=True, save_format="excel")

Or to save your outputs as a `.geojson` file which can be loaded into GIS software (e.g QGIS), you can use the `save_patches_to_geojson()` method.

For now, this will just contain all the information in your patch dataframe but will eventually contain all the patch predictions and confidence scores too.

In [ ]:
my_files.save_patches_to_geojson("test_patches.geojson")

-----

# Documentation

Please refer to the [MapReader documentation](https://mapreader.readthedocs.io/en/latest/) for more information.

---------

Now head to part 2 of the workshop notebooks!